In [12]:
# Install necessary libraries
# !pip install torch torchvision matplotlib opencv-python tqdm gym numpy

# Import required libraries
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
from torchvision.ops import nms
import numpy as np
import cv2
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
import random
import glob
import gym
from collections import deque

In [13]:
class UCSDAnomalyDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = sorted(glob.glob(os.path.join(root_dir, "Test*/", "*.tif")))  # Load all test folders

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("L")  # Convert to grayscale
        if self.transform:
            image = self.transform(image)
        return image, img_path

# Define transformations
transform = transforms.Compose([
    transforms.Resize((256, 256)),  
    transforms.ToTensor(),          
    transforms.Normalize(mean=[0.5], std=[0.5])  
])

# Load dataset (All test files)
dataset_path = "/kaggle/input/ucsd-anomaly-detection-dataset/UCSD_Anomaly_Dataset.v1p2/UCSDped2/Test"
dataset = UCSDAnomalyDataset(root_dir=dataset_path, transform=transform)
dataloader = DataLoader(dataset, batch_size=500, shuffle=False, num_workers=4)

print(f"Loaded dataset with {len(dataset)} frames from all test files")


Loaded dataset with 2010 frames from all test files


In [14]:
# Feature extraction
resnet = models.resnet152(pretrained=True)
resnet.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
feature_extractor = nn.Sequential(*list(resnet.children())[:-2])
feature_extractor.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
feature_extractor = feature_extractor.to(device)

def extract_features(images):
    images = images.to(device)
    with torch.no_grad():
        features = feature_extractor(images)
    return features
# 

In [15]:
# region proposal network
class RPN(nn.Module):
    def __init__(self, in_channels=2048, mid_channels=512, num_anchors=9):
        super(RPN, self).__init__()
        self.conv = nn.Conv2d(in_channels, mid_channels, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.cls_logits = nn.Conv2d(mid_channels, num_anchors * 2, kernel_size=1)  
        self.bbox_pred = nn.Conv2d(mid_channels, num_anchors * 4, kernel_size=1)  

    def forward(self, x):
        x = self.relu(self.conv(x))
        logits = self.cls_logits(x)
        bbox_preds = self.bbox_pred(x)
        return logits, bbox_preds

rpn = RPN().to(device)

In [16]:
class DQN(nn.Module):
    def __init__(self, input_size=2048 * 8 * 8, output_size=2):  # Fix input size
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

# Initialize DQN with the correct input size
dqn = DQN(input_size=2048 * 8 * 8, output_size=2).to(device)


optimizer_dqn = optim.Adam(dqn.parameters(), lr=0.001)
criterion_dqn = nn.MSELoss()
replay_memory = deque(maxlen=10000)

gamma = 0.95  
epsilon = 1.0  
epsilon_min = 0.01  
epsilon_decay = 0.995  


In [20]:
def train_rpn_dqn():
    rpn.train()
    dqn.train()

    optimizer_rpn = optim.SGD(rpn.parameters(), lr=0.05, momentum=0.9)
    optimizer_dqn = optim.Adam(dqn.parameters(), lr=0.001)

    criterion_dqn = nn.MSELoss()  # Ensure loss function is properly defined

    for epoch in range(15):  
        epoch_loss_rpn = 0
        epoch_loss_dqn = 0
        correct_rpn = 0
        total_rpn = 0
        correct_dqn = 0
        total_dqn = 0

        for images, _ in tqdm(dataloader, desc=f"Epoch {epoch + 1}/15"):
            images = images.to(device)
            features = extract_features(images)

            # === RPN Forward Pass ===
            logits, bbox_preds = rpn(features)  
            batch_size, num_anchors_x2, H, W = logits.shape
            num_anchors = num_anchors_x2 // 2  

            # Generate Random Targets (For Testing)
            targets_cls = torch.randint(0, 2, (batch_size, num_anchors, H, W), device=device)  
            targets_bbox = torch.randn_like(bbox_preds, device=device)  

            logits = logits.permute(0, 2, 3, 1).reshape(-1, 2)  
            targets_cls = targets_cls.permute(0, 2, 3, 1).reshape(-1).long()  

            # Compute RPN Loss
            cls_loss = nn.CrossEntropyLoss()(logits, targets_cls)  
            bbox_loss = nn.SmoothL1Loss()(bbox_preds, targets_bbox)
            loss_rpn = cls_loss + bbox_loss

            optimizer_rpn.zero_grad()  # 
            loss_rpn.backward()
            optimizer_rpn.step()
            
            epoch_loss_rpn += loss_rpn.item()

            # === RPN Accuracy Calculation ===
            predicted_classes = torch.argmax(logits, dim=1)
            correct_rpn += (predicted_classes == targets_cls).sum().item()
            total_rpn += targets_cls.numel()

            # === DQN Training ===
            state = features.view(features.size(0), -1).to(device)  
            q_values = dqn(state)
            predicted_actions = torch.argmax(q_values, dim=1)

            # Generate Random Labels (For Testing)
            true_labels = torch.randint(0, 2, (q_values.shape[0],), device=device)

            loss_dqn = criterion_dqn(q_values, nn.functional.one_hot(true_labels, num_classes=2).float())

            optimizer_dqn.zero_grad()  
            loss_dqn.backward()
            optimizer_dqn.step()

            epoch_loss_dqn += loss_dqn.item()

            # === DQN Accuracy Calculation ===
            correct_dqn += (predicted_actions == true_labels).sum().item()
            total_dqn += true_labels.numel()

        # Compute Final Accuracy
        rpn_accuracy = (correct_rpn / total_rpn) * 100
        dqn_accuracy = (correct_dqn / total_dqn) * 100

        print(f"Epoch {epoch + 1}:")
        print(f"   - RPN Loss = {epoch_loss_rpn:.4f}, Accuracy = {rpn_accuracy:.2f}%")
        print(f"   - DQN Loss = {epoch_loss_dqn:.4f}, Accuracy = {dqn_accuracy:.2f}%")

# Train the model and track accuracy
train_rpn_dqn()


Epoch 1/15: 100%|██████████| 5/5 [00:19<00:00,  3.98s/it]


Epoch 1:
   - RPN Loss = 5.5939, Accuracy = 49.99%
   - DQN Loss = 99.4157, Accuracy = 50.45%


Epoch 2/15: 100%|██████████| 5/5 [00:19<00:00,  3.94s/it]


Epoch 2:
   - RPN Loss = 5.5868, Accuracy = 50.03%
   - DQN Loss = 45.2924, Accuracy = 49.90%


Epoch 3/15: 100%|██████████| 5/5 [00:20<00:00,  4.11s/it]


...

Epoch 13:
   - RPN Loss = 5.5916, Accuracy = 50.02%
   - DQN Loss = 1.3238, Accuracy = 49.90%


Epoch 14/15: 100%|██████████| 5/5 [00:20<00:00,  4.01s/it]


Epoch 14:
   - RPN Loss = 5.5881, Accuracy = 49.98%
   - DQN Loss = 1.2856, Accuracy = 49.15%


Epoch 15/15: 100%|██████████| 5/5 [00:20<00:00,  4.03s/it]

Epoch 15:
   - RPN Loss = 5.5892, Accuracy = 49.94%
   - DQN Loss = 1.2861, Accuracy = 51.29%
